## EEP153 Project 0 - Team Greville

### Import necessary libraries

In [1]:
!pip install wbdata
import wbdata
import numpy as np
import pandas as pd
import cufflinks as cf
cf.go_offline()

  Using cached wbdata-0.3.0-py3-none-any.whl (14 kB)
  Using cached tabulate-0.8.7-py3-none-any.whl (24 kB)


/opt/conda/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning:

The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.

Error importing optional module geopandas
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/_plotly_utils/optional_imports.py", line 30, in get_module
    return import_module(name)
  File "/opt/conda/lib/python3.8/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1014, in _gcd_import
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>

### Create population function 
        
Takes in a given year, population parameter ("Male", "Female", "Everyone"), the minimum age, the maximum age, and the desired country (ex "USA" or "WLD" for world) an returns the counts of the given age groups aggregated in 5 year intervals

In [2]:
def population(year, population, low, high, location):
    age_ranges = []
    if high > 80:
        for i in range(low,80,5):
            age_ranges.append(f"{i:02d}"+f"{i+4:02d}")
        age_ranges.append("80UP")
    else:
        if high % 5 == 0:
            for i in range(low,high,5):
                age_ranges.append(f"{i:02d}"+f"{i+4:02d}")
        else:
            for i in range(low,high,5):
                age_ranges.append(f"{i:02d}"+f"{i+4:02d}")
    
    if population == "Male" or population == "Males":
        male_variables = {"SP.POP."+age_range+".MA":"Males "+age_range for age_range in age_ranges}
        variables = male_variables
    elif population == "Female" or population == "Females":
        female_variables = {"SP.POP."+age_range+".FE":"Females "+age_range for age_range in age_ranges}
        variables = female_variables
    else:
        male_variables = {"SP.POP."+age_range+".MA":"Males "+age_range for age_range in age_ranges}
        female_variables = {"SP.POP."+age_range+".FE":"Females "+age_range for age_range in age_ranges}
        variables = male_variables
        variables.update(female_variables)
        
    data = wbdata.get_dataframe(variables,country=location)
    data.reset_index(inplace=True)
    data['date'] = data['date'].astype(int)
    data.set_index(['date'],inplace=True)
    df = data.query("date=="+str(year))
    date_str = "date==" + str(year)
    print(df.query(date_str).sum(axis=0))
    
population(2008, "All", 15, 85, "GBR")

Males 1519      2055526.0
Males 2024      2107117.0
Males 2529      2029664.0
Males 3034      1993729.0
Males 3539      2167269.0
Males 4044      2329149.0
Males 4549      2183220.0
Males 5054      1877733.0
Males 5559      1834642.0
Males 6064      1764858.0
Males 6569      1353073.0
Males 7074      1132283.0
Males 7579       894554.0
Males 80UP      1020399.0
Females 1519    1972199.0
Females 2024    2067185.0
Females 2529    2028566.0
Females 3034    2011134.0
Females 3539    2207305.0
Females 4044    2382044.0
Females 4549    2228958.0
Females 5054    1909715.0
Females 5559    1879547.0
Females 6064    1830024.0
Females 6569    1448740.0
Females 7074    1276464.0
Females 7579    1108272.0
Females 80UP    1810474.0
dtype: float64


### Create population_df function 
        
Takes in a given year and the desired country (ex "USA" or "WLD" for world) and returns a one row dataframe of the counts of all ages aggreagted in 5 year groups

In [3]:
def population_df(year, location):
    age_ranges = []
    for i in range(0,80,5):
        age_ranges.append(f"{i:02d}"+f"{i+4:02d}")
    age_ranges.append("80UP")

    male_variables = {"SP.POP."+age_range+".MA":"Males "+age_range for age_range in age_ranges}
    female_variables = {"SP.POP."+age_range+".FE":"Females "+age_range for age_range in age_ranges}
    variables = male_variables
    variables.update(female_variables)
        
    df = wbdata.get_dataframe(variables,country="WLD")
    #final_df = pd.DataFrame(df, columns=["Population Age Range", "Population Count"])
    return df.loc[[str(year)],:]

population_df(2008, "GBR")

,Males 0004,Males 0509,Males 1014,Males 1519,Males 2024,Males 2529,Males 3034,Males 3539,Males 4044,Males 4549,...,Females 3539,Females 4044,Females 4549,Females 5054,Females 5559,Females 6064,Females 6569,Females 7074,Females 7579,Females 80UP
date,,,,,,,,,,,,,,,,,,,,,
2008,330870369.0,316287094.0,309906090.0,313386351.0,304771450.0,270286669.0,253155153.0,245625510.0,225156305.0,195970798.0,...,239607155.0,221241964.0,193690900.0,170498344.0,146017181.0,110758552.0,89109625.0,74622378.0,54260962.0,62058514.0


### Construct setup variables for population_pyramid
        
Creates a list of possible age ranges aggreagted at 5 year interval with the final group being 80 years and over, as well as creating the necessary male and female variables for the API

In [4]:
age_ranges = []
for i in range(0,80,5):
    age_ranges.append(f"{i:02d}"+f"{i+4:02d}")

age_ranges.append("80UP")

male_variables = {"SP.POP."+age_range+".MA":"Males "+age_range for age_range in age_ranges}
female_variables = {"SP.POP."+age_range+".FE":"Females "+age_range for age_range in age_ranges}

variables = male_variables
variables.update(female_variables)

df = wbdata.get_dataframe(variables,country="GBR")
df

,Males 0004,Males 0509,Males 1014,Males 1519,Males 2024,Males 2529,Males 3034,Males 3539,Males 4044,Males 4549,...,Females 3539,Females 4044,Females 4549,Females 5054,Females 5559,Females 6064,Females 6569,Females 7074,Females 7579,Females 80UP
date,,,,,,,,,,,,,,,,,,,,,
2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019,2002151.0,2083573.0,1964877.0,1860465.0,2074980.0,2261807.0,2319836.0,2222680.0,2107164.0,2131817.0,...,2250685.0,2119619.0,2177219.0,2349587.0,2243179.0,1911026.0,1730542.0,1716036.0,1242301.0,2003644.0
2018,2015968.0,2076882.0,1917793.0,1873417.0,2095259.0,2268214.0,2297014.0,2188332.0,2092905.0,2168240.0,...,2211818.0,2109059.0,2219689.0,2358858.0,2190603.0,1858746.0,1770028.0,1654634.0,1209696.0,1991688.0
2017,2031768.0,2060487.0,1867936.0,1894805.0,2112753.0,2270083.0,2269216.0,2154975.0,2085423.0,2209684.0,...,2171731.0,2107285.0,2266416.0,2355574.0,2129646.0,1814041.0,1819689.0,1573916.0,1190379.0,1977415.0
2016,2049855.0,2030476.0,1824762.0,1915762.0,2127340.0,2265635.0,2237577.0,2124331.0,2087212.0,2246486.0,...,2135432.0,2115294.0,2307100.0,2338474.0,2066638.0,1785700.0,1853528.0,1495948.0,1177005.0,1958023.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2424299.0,2038602.0,1975872.0,2135784.0,1756696.0,1674812.0,1673939.0,1751103.0,1828729.0,1629697.0,...,1748410.0,1862950.0,1695815.0,1866507.0,1789035.0,1619527.0,1364130.0,1095953.0,812072.0,780908.0
1963,2388729.0,1967594.0,2043087.0,2073240.0,1724941.0,1661051.0,1678312.0,1793079.0,1780592.0,1665763.0,...,1798093.0,1819463.0,1739030.0,1874658.0,1776188.0,1592256.0,1341530.0,1084094.0,796477.0,768176.0
1962,2333842.0,1916887.0,2115562.0,1984936.0,1696996.0,1653593.0,1682542.0,1834635.0,1719962.0,1715725.0,...,1845988.0,1764443.0,1796954.0,1869410.0,1760713.0,1564118.0,1322460.0,1073873.0,780927.0,747601.0


### Create population_pyramid function
        
Creates a population pyramid using a dataframe corresponding to a given year and country or the world population

In [5]:
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)


def population_pyramid(dataframe):
    layout = go.Layout(barmode='overlay',
                   yaxis=go.layout.YAxis(range=[0, 90], title='Age'),
                   xaxis=go.layout.XAxis(title='Number'))

    year = dataframe.index[0]

    bins = [go.Bar(x = dataframe.loc[str(year),:].filter(regex="Male").values,
               y = [int(s[:2])+1 for s in age_ranges],
               orientation='h',
               name='Men',
               marker=dict(color='purple'),
               hoverinfo='skip'
               ),

    go.Bar(x = -dataframe.loc[str(year),:].filter(regex="Female").values,
               y=[int(s[:2])+1 for s in age_ranges],
               orientation='h',
               name='Women',
               marker=dict(color='pink'),
               hoverinfo='skip',
               )
        ]
    py.iplot(dict(data=bins, layout=layout))
    
population_pyramid(population_df(2008, "GBR"))

### Construct animated_population_pyramid function
        
Takes in a dataframe and creates an animated population pyramid showing the change in population over time in each decade from 1960 to 2018

In [6]:

def animated_population_pyramid(dataframe):

    # Count down by increments of 20 years
    years = range(2018,1960,-10)
    
    layout = go.Layout(barmode='overlay',
                   yaxis=go.layout.YAxis(range=[0, 90], title='Age'),
                   xaxis=go.layout.XAxis(title='Number'))

    # This makes a list of graphs, year by year
    bins = [go.Bar(x = df.loc[str(year),:].filter(regex="Male").values,
               y = [int(s[:2])+1 for s in age_ranges],
               orientation='h',
               name='Men {:d}'.format(year),
               hoverinfo='skip'
              )
            for year in years]
          
    bins += [go.Bar(x = -df.loc[str(year),:].filter(regex="Female").values,
                y=[int(s[:2])+1 for s in age_ranges],
                orientation='h',
                name='Women {:d}'.format(year),
                hoverinfo='skip',
               )
             for year in years]

    return py.iplot(dict(data=bins, layout=layout))
         
animated_population_pyramid(df)